In [1]:
import pandas as pd
import numpy as np
import os
import json
import altair as alt
import numpy as np
import glob

In [6]:
DATA_DIR = "/Users/user/Documents/GeneInvestigator/results/BDNF/Recombinants"

RELAX_FILES = glob.glob(os.path.join(DATA_DIR, "*.RELAX.json"))
pvalue_threshold = 0.05

In [7]:
RELAX_FILES

['/Users/user/Documents/GeneInvestigator/results/BDNF/Recombinants/BDNF_codons_RDP_recombinationFree.fas.Carnivora.RELAX.json',
 '/Users/user/Documents/GeneInvestigator/results/BDNF/Recombinants/BDNF_codons_RDP_recombinationFree.fas.Artiodactyla.RELAX.json',
 '/Users/user/Documents/GeneInvestigator/results/BDNF/Recombinants/BDNF_codons_RDP_recombinationFree.fas.Chiroptera.RELAX.json',
 '/Users/user/Documents/GeneInvestigator/results/BDNF/Recombinants/BDNF_codons_RDP_recombinationFree.fas.Primates.RELAX.json',
 '/Users/user/Documents/GeneInvestigator/results/BDNF/Recombinants/BDNF_codons_RDP_recombinationFree.fas.Glires.RELAX.json',
 '/Users/user/Documents/GeneInvestigator/results/BDNF/Recombinants/BDNF_codons_RDP_recombinationFree.fas.Eulipotyphla.RELAX.json',
 '/Users/user/Documents/GeneInvestigator/results/BDNF/Recombinants/BDNF_codons_RDP_recombinationFree.fas.Perissodactyla.RELAX.json']

In [8]:
def getRELAX_TR(JSON):
    with open(JSON, "r") as in_d:
        json_data = json.load(in_d)
    return json_data["test results"]

def getRELAX_fits(JSON):
    with open(JSON, "r") as in_d:
        json_data = json.load(in_d)
    return json_data["fits"]


In [18]:
# Main
passedThreshold = []

for file in RELAX_FILES:
    pval = getRELAX_TR(file)["p-value"]
    if pval <= pvalue_threshold:
        passedThreshold.append(file)
        print("# Passed p-value threshold:", file.split("/")[-1])


# Passed p-value threshold: BDNF_codons_RDP_recombinationFree.fas.Carnivora.RELAX.json
# Passed p-value threshold: BDNF_codons_RDP_recombinationFree.fas.Primates.RELAX.json
# Passed p-value threshold: BDNF_codons_RDP_recombinationFree.fas.Eulipotyphla.RELAX.json


In [73]:
# Process them separately

#TEST_FILE = "/Users/user/Documents/GeneInvestigator/results/BDNF/Recombinants/BDNF_codons_RDP_recombinationFree.fas.Carnivora.RELAX.json"
#TEST_FILE = "/Users/user/Documents/GeneInvestigator/results/BDNF/Recombinants/BDNF_codons_RDP_recombinationFree.fas.Primates.RELAX.json"
TEST_FILE = '/Users/user/Documents/GeneInvestigator/results/BDNF/Recombinants/BDNF_codons_RDP_recombinationFree.fas.Eulipotyphla.RELAX.json'

label = TEST_FILE.split("/")[-1].split(".")[2]
print("Label:", label)
fits = getRELAX_fits(TEST_FILE)
fits.keys()

# print(fits["RELAX partitioned descriptive"])

Label: Eulipotyphla


dict_keys(['General descriptive', 'MG94xREV with separate rates for branch sets', 'Nucleotide GTR', 'RELAX alternative', 'RELAX null', 'RELAX partitioned descriptive'])

In [74]:
df = pd.DataFrame.from_dict(fits, orient='index')
df.T

,General descriptive,MG94xREV with separate rates for branch sets,Nucleotide GTR,RELAX alternative,RELAX null,RELAX partitioned descriptive
AIC-c,28189.340578,28889.487503,30929.613123,28126.233245,28136.38249,28132.656357
Log Likelihood,-13577.18239,-14180.567018,-15195.397457,-13794.903809,-13800.987503,-13794.078738
Rate Distributions,{'Shared': {'0': {'omega': 6.057267936146019e-...,{'non-synonymous/synonymous rate ratio for *Re...,{'Substitution rate from nucleotide A to nucle...,{'Reference': {'0': {'omega': 5.48720376094430...,{'Reference': {'0': {'omega': 0.02288401406824...,{'Reference': {'0': {'omega': 0.01943261683843...
display order,4,1,0,2,3,5
estimated parameters,513,263,269,267,266,271
Equilibrium frequencies,NaN,"[[0.01795196462709524], [0.03032011369858167],...","[[0.259677450432571], [0.2464594693539374], [0...",NaN,NaN,NaN


In [75]:
df2_Ref = pd.DataFrame.from_dict(fits["RELAX partitioned descriptive"]["Rate Distributions"]["Reference"], orient='index')
df2_Ref

,omega,proportion
0,0.019433,0.835757
1,0.449135,0.157988
2,141.510056,0.006255


In [76]:
df3_Test = pd.DataFrame.from_dict(fits["RELAX partitioned descriptive"]["Rate Distributions"]["Test"], orient='index')
df3_Test

,omega,proportion
0,0.102218,0.872006
1,0.667602,0.127994
2,12.103414,0.000000


In [77]:
# Unconstrained model

source = df2_Ref
df2_Ref["log(omega)"] = np.log10(df2_Ref["omega"])
df2_Ref["Color"] = "Reference"

line = alt.Chart(source).mark_bar().encode(
    x='log(omega)',
    y='proportion',
    color="Color"
).properties(
    width=500,
    height=400,
    title=label
)

line.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

df3_Test["log(omega)"] = np.log10(df3_Test["omega"])
df3_Test["Color"] = "Test"

source = df3_Test

line2 = alt.Chart(source).mark_bar().encode(
    x='log(omega)',
    y='proportion',
    color="Color"
).properties(
    width=500,
    height=400)



line + line2

alt.LayerChart(...)

In [20]:
# Process them together as subplots

In [ ]:
# RELAX Interpretation
# http://hyphy.org/methods/selection-methods/#relax
# method paper: https://academic.oup.com/mbe/article/32/3/820/981440



# Figure legend
Patterns of natural selection across taxonomic groups under the Partitioned Descriptive model of the RELAX method. Selection profiles for BDNF are shown along Reference and Test branches for each taxonomic group. Three omega parameters and the relative proportion of sites they represent are plotted for Test (orange) and Referenc (blue) branches. Only omega categories representing nonzero proportions of sites are shown. Neutral selection corresponds to the omega=1.0 in this log10 scaled X-axis. These taxonomic groups represent datasets where significant (p<= 0.05) for relaxed selection was detected between test and reference branches.